In [1]:
# Rolling hash

In [ ]:
# 28. Implement strStr()
def strStr(haystack, needle):
    """
    :type haystack: str
    :type needle: str
    :rtype: int
    """
    hlen, nlen = len(haystack),len(needle)
    if nlen==0:
        return 0
    if hlen<nlen:
        return -1

    base = 31
    mod = 10**9+7
    rolling = lambda x,y:x*base+y
    get_hash = lambda a:ord(a)-ord('a')

    nhash = reduce(rolling, map(get_hash, needle))%mod
    hhash = reduce(rolling, map(get_hash, haystack[:nlen]))%mod
    if nhash == hhash and needle == haystack[:nlen]:
        return 0

    high_base = (base**(nlen-1)) % mod
    for i in range(nlen,hlen):
        hhash = hhash - get_hash(haystack[i-nlen])*high_base
        hhash = (hhash*base+get_hash(haystack[i]))%mod
        if hhash==nhash and needle == haystack[i-nlen+1:i+1]:
            return i-nlen+1
    return -1

In [2]:
# # 1392. Longest Happy Prefix,最长的prefix前后一致
def longestPrefix(s):
    """
    :type s: str
    :rtype: str
    """
    base, mod = 31, 10**9+7
    get_hash = lambda a:ord(a)-ord('a')
    pre, suf = 0,0

    res = -1
    for i in range(len(s)-1):
        pre = (pre*base + get_hash(s[i]))%mod
        suf = (suf + get_hash(s[-i-1])*pow(base,i,mod))%mod
        if pre==suf and s[:i+1]==s[-i-1:]:
            res = i

    return s[:res+1]

In [1]:
# 1147. Longest Chunked Palindrome Decomposition，将string分割成多块回文
def longestDecomposition(text):
    """
    :type text: str
    :rtype: int
    """
    base,mod = 31,10**9+7

    get_hash = lambda x:ord(x)-ord('a')
    pre,suf = 0,0

    res = 0
    index,curbase = 0,1
    while index<len(text)//2:
        pre = (pre*base+get_hash(text[index]))%mod
        suf = (suf+get_hash(text[-index-1])*curbase)%mod
        curbase = curbase*base
        if pre==suf:
            res+=1
            pre,suf,curbase = 0,0,1
        index+=1
    if pre==suf:
        return res*2+len(text)%2
    else:
        return res*2+1
    
def longestDecomposition(text):
    """
    :type text: str
    :rtype: int
    """
    res = 0
    left,right = "",""
    for i,j in zip(text,text[::-1]):
        left = left+i
        right = j+right
        if left==right:
            res+=1
            left,right = "",""
    return res

longestDecomposition(text = "ghiabcdefhelloadamhelloabcdefghi")

7

In [7]:
# 1044. Longest Duplicate Substring,寻找最长的重复substring（可重叠）

# Binary Search in range 1 and N, so it's O(logN)
# Rolling hash O(N)
# Overall O(NlogN)
# SpaceO(N)

from functools import reduce
def longestDupSubstring(S):
    """
    :type S: str
    :rtype: str
    """
    base,mod = 31,10**9+7

    get_hash = lambda x:ord(x)-ord('a')
    rolling = lambda x,y:x*base+y

    def isExist(n,S):
        visited = dict()
        curhash = reduce(rolling,map(get_hash,S[:n]))%mod
        visited[curhash]=0
        high_base = pow(base,n-1,mod)
        for i in range(n,len(S)):
            curhash = curhash - high_base*get_hash(S[i-n])
            curhash = (curhash*base+get_hash(S[i]))%mod
            if curhash in visited and S[visited[curhash]:visited[curhash]+n]==S[i-n+1:i+1]:
                return (True,i-n+1)
            visited[curhash] = i-n+1
        return (False,None)

    left,right,res = 1,len(S),[0,0]
    while left<right:
        mid = left+(right-left)//2
        flag,word = isExist(mid,S)
        if flag:#如果mid能找到
            left = mid+1
            if mid>res[1]-res[0]:
                res = [word,word+mid]
        else:#如果不存在比这更长的
            right = mid
    return S[res[0]:res[1]]

def longestDupSubstring(S):
    """
    :type S: str
    :rtype: str
    """
    A = [ord(c) - ord('a') for c in S]
    mod = 2**63 - 1

    def test(L):
        p = pow(26, L, mod)
        cur = reduce(lambda x, y: (x * 26 + y) % mod, A[:L], 0)
        seen = {cur}
        for i in range(L, len(S)):
            cur = (cur * 26 + A[i] - A[i - L] * p) % mod
            if cur in seen: return i - L + 1
            seen.add(cur)
    res, lo, hi = 0, 0, len(S)
    while lo < hi:
        mi = (lo + hi + 1) // 2
        pos = test(mi)
        if pos:
            lo = mi
            res = pos
        else:
            hi = mi - 1
    return S[res:res + lo]

longestDupSubstring("banana")

'ana'

In [1]:
# 214. Shortest Palindrome，在字符串前面添加一部分，使得回文的最短长度
# 即为前面这一部分的正反rolling hash值相等，不断添加直到相等

def shortestPalindrome(s):
    """
    :type s: str
    :rtype: str
    """
    base,mod = 31,10**9+7
    prefix,suffix = 0,0
    for index,c in enumerate(s):#the prefix of original string
        prefix = (prefix * base + ord(c)-ord('a'))%mod
    for index,c in enumerate(s[::-1]):#the suffix of reverse
        suffix = (suffix * base + ord(c)-ord('a'))%mod
    if prefix==suffix and s == s[::-1]:
        return s

    for i in range(len(s)):
        prefix = (prefix-(ord(s[-i-1])-ord('a'))*pow(base,i,mod))%mod #选取片段为0:-i+1
        suffix = (suffix*base - (ord(s[-i-1])-ord('a'))*pow(base,len(s),mod))%mod#选取片段为i+1:
        if prefix==suffix and s[:-(i+1)] == s[:-(i+1)][::-1]:
            return s[-(i+1):][::-1]+s
    return s[::-1]+s

def shortestPalindrome(s):
    """
    :type s: str
    :rtype: str
    """
    if not s:
        return ""
    base,mod = 31,10**9+7
    prefix,suffix = 0,0
    for index,c in enumerate(s):
        prefix = (prefix * base + ord(c)-ord('a'))%mod#the prefix of original string
        suffix = (suffix + (ord(c)-ord('a'))*pow(base,index,mod))%mod#the suffix of reversed string
        if prefix==suffix:
            curlen = index
    return s[curlen+1:][::-1]+s
    
shortestPalindrome("aacdaa")

'aadcaacdaa'

In [31]:
# 1554. Strings Differ by One Character，长度一样的字符串，是否存在两个字符串，只相差一个元素
def differByOne(self, dict):
    """
    :type dict: List[str]
    :rtype: bool
    """
    base,mod = 26,10**11+7
    # 使用31, 10**9+7时出现了collision
    visited = set()
    level = []
    for s in dict:
        cur = 0
        for i,c in enumerate(s):
            cur = (base*cur+ord(c)-ord('a'))%mod
        level.append(cur)
        visited.add(cur)

    for index,s in enumerate(dict):
        cur = level[index]
        curbase = 1
        for i in range(len(s)):
            c = s[len(s)-i-1]
            for t in "abcdefghijklmnopqrstuvwxyz":
                if t==c:
                    continue
                cur2 = (cur+(ord(t)-ord(c))*curbase)%mod
                if cur2 in visited:
                    return True
            curbase = curbase*base%mod
    return False

def differByOne(dict):
    hset = set()
    for word in dict:
        for i in range(len(word)):
            combo = word[:i] + '*' + word[i+1:]
            if combo in hset:
                return True
            hset.add(combo)
    return False

differByOne(dict = ["abcd","acbd", "aacd"])

True